In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import scipy as sp
import scipy.stats
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools
import re
res_digit = r'[0-9]'

# fourier transform
from scipy.fft import fft, ifft

from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.manifold import TSNE



In [ ]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")



In [ ]:
## ensure relative path to data directory is sound
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

from predicament.utils.config import FEATURED_BASE_PATH

In [ ]:
from predicament.data.timeseries import create_participant_data_edf_only
from predicament.data.windowed import window_all_participants_data
from predicament.data.windowed import merge_condition_data
from predicament.data.partitioning import between_subject_cv_partition

from predicament.data.features import MAXIMAL_FEATURE_GROUP
from predicament.data.features import STATS_FEATURE_GROUP
from predicament.data.features import INFO_FEATURE_GROUP
from predicament.data.features import FREQ_FEATURE_GROUP
from predicament.data.features import convert_timeseries_to_features
from prepare_evaluation_data import load_dataframe_and_config


from predicament.evaluation.balancing import get_group_label_counts
from predicament.evaluation.balancing import balance_data

from predicament.evaluation.results import output_model_best_from_results
from predicament.evaluation.results import save_results_df_to_file


In [ ]:
from predicament.evaluation.results import test
from predicament.evaluation.results import save_results_df_to_file


## Load features

Before running this, you will need to generate featured data. See README file for details. For the variable `subdir` below replace this with the subdirectory name of the featured data.

In [ ]:
subdir = '20231129210920'
featured_data_dir = os.path.join(FEATURED_BASE_PATH,subdir)

featured_df, featured_config = load_dataframe_and_config(
    featured_data_dir, 'featured.csv')
n_channels = int(featured_config['LOAD']['n_channels'])
channels = json.loads(featured_config['LOAD']['channels'].replace("'",'"'))
participant_list = json.loads(featured_config['LOAD']['participant_list'].replace("'",'"'))
Fs = int(featured_config['LOAD']['sample_rate'])
window_size = int(featured_config['LOAD']['window_size'])
time = window_size/Fs
print(f"Fs: {Fs}, n_samples = {window_size}, time: {time}s, n_channels: {n_channels}")


In [ ]:
featured_df.columns

In [ ]:
# balance featured data
subject_condition_counts = get_group_label_counts(featured_df, 'participant', 'condition')
print(f"before balancing: subject_condition_counts = {subject_condition_counts}")
featured_df = balance_data(featured_df, group_col='participant', label_col='condition')
subject_condition_counts = get_group_label_counts(featured_df, 'participant', 'condition')

print(f"after balancing: subject_condition_counts = {subject_condition_counts}")

## Classification

In [ ]:
import pandas as pd

from sklearn.model_selection import RandomizedSearchCV 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer


In [ ]:
# model name
name = 'GradientBoosting'

# choose which search to perform
# search_type = 'random_search'
search_type = 'bayesian_optimization'


# you can choose a subset of the feature types to use
features_to_use = set(
    ['Mean', 'SD', 'MAD', 'Max', 'Min', 'Energy', 'IQR',
     'arCoeff', 'Correlation', 'MaxFreqInd', 'MeanFreq', 'FreqSkewness',
    'FreqKurtosis'
    ])
# this constructs a list of the featured data columns based on the above feature types
columns_to_use = [ col for col in featured_df.columns if re.sub(res_digit, '', col) in features_to_use ]
print(f"columns_to_use = {columns_to_use}")

designmtx = featured_df[columns_to_use].to_numpy()

condition_data = featured_df['condition'].values.astype(int)

## Hold one group out

In [ ]:
# standard cross-validation
# Match all digits in the string and replace them with an empty string
# new_string = re.sub(pattern, '', string1)
subjects = np.unique(featured_df['participant'])
n_subjects = len(subjects)
groups = np.empty(len(featured_df), dtype=int)
for s, sub in enumerate(subjects):
    groups[featured_df['participant']==sub] = s
# cross validation splits    
group_kfold = GroupKFold(n_splits=n_subjects)
# number of iterations for your search
n_iter = 10

## Random Search

Here we use a random grid to search for best hyperparameters


In [ ]:
if search_type == 'random_search':
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 10)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False] 
    # Create the random grid
    random_grid = {
        'n_estimators': n_estimators,
        'max_features': max_features,
        'max_depth': max_depth,
        'min_samples_split': min_samples_split,
        'min_samples_leaf': min_samples_leaf,
        'bootstrap': bootstrap}

    # First create the base model to tune
    estimator = RandomForestClassifier()
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    param_search = RandomizedSearchCV(
        estimator = estimator,
        param_distributions = random_grid,
        n_iter = n_iter,
        cv=group_kfold,
        verbose=2,
        random_state=42,
        n_jobs = -1)

    # Fit the random search model
    _ = param_search.fit(X=designmtx, y=condition_data, groups=groups)


## Bayesian Optimisation
May be quicker and more effective than random search

In [ ]:
# Bayesian optimisation

if search_type == 'bayesian_optimization':

    # this is just a naive initial guess at what will work
    # note that the log-uniform prior helps to focus on 
    # smaller values
    search_spaces = dict(
        learning_rate = Real(1e-6, 5e-1, prior='log-uniform'),
        n_estimators = Integer(10,200, prior='log-uniform'),
        subsample = Real(1e-10, 1, prior='uniform'),
#         criterion = Categorical(['friedman_mse', 'squared_error']),
        #min_weight_fraction_leaf=0.0,
        max_depth = Integer(1, 100,  prior='log-uniform'),
        # min_impurity_decrease=0.0,
        max_features = Real(1e-1, 1, prior='log-uniform'),
        # max_leaf_nodes=None, 
        #ccp_alpha = 
    )

    estimator = GradientBoostingClassifier()
    # log-uniform: understand as search over p = exp(x) by varying x
    param_search = BayesSearchCV(
        estimator, search_spaces,
        cv=group_kfold, verbose=3, random_state=42, n_iter=n_iter,
        n_jobs = -1)

    # executes bayesian optimization
    _ = param_search.fit(X=designmtx, y=condition_data, groups=groups)


In [ ]:
result_df = pd.DataFrame(param_search.cv_results_)
result_df.insert(0, 'model', name)
result_df.insert(1, 'held out', 'subject')
result_df.insert(2, 'feature set', str(features_to_use))
display(result_df)
save_results_df_to_file(result_df, f'{search_type}_{name}')

In [ ]:
output_model_best_from_results(result_df)

In [ ]:
estimator.get_params()